# Fusion Localization Visualization

融合定位中间变量可视化

In [ ]:
import os
import re
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Data Loading

In [ ]:
log_file_path = "./tracking_control_node-2-stdout.log"

TUPLE_PATTERN = re.compile(
    r"(?P<key>state|z) = \((?P<value>[^)]+)\)"
)
FLOAT_PATTERNS = (
    re.compile(r"(?P<key>gnss_X)=(?P<value>-?\d*(?:\.\d*)?)"),
    re.compile(r"(?P<key>gnss_Y)=(?P<value>-?\d*(?:\.\d*)?)"),
    re.compile(r"(?P<key>enu_x)=(?P<value>-?\d*(?:\.\d*)?)"),
    re.compile(r"(?P<key>enu_y)=(?P<value>-?\d*(?:\.\d*)?)"),
    re.compile(r"(?P<key>enu_z)=(?P<value>-?\d*(?:\.\d*)?)"),
)

data_dict = defaultdict(list)
with open(log_file_path, "r", encoding="utf-8") as log_file:
    for line in log_file:
        if match_result := TUPLE_PATTERN.search(line):
            key = match_result.group("key")
            value = tuple(
                float(s.strip())
                for s in match_result.group("value").split(',')
            )
            data_dict[key].append(value)
        for pattern in FLOAT_PATTERNS:
            if match_result := pattern.search(line):
                key = match_result.group("key")
                value = float(match_result.group("value"))
                data_dict[key].append(value)

{ key: len(value) for key, value in data_dict.items() }

In [ ]:
data_list = [
    {key: data_dict[key][i] for key in data_dict.keys()}
    for i in range(len(data_dict["state"]))
]
heading = np.array([item["state"][2] for item in data_list])

transponder_correction_filter = np.array([
    (item["z"][0] != item["state"][0]) or (item["z"][1] != item["state"][1])
    for item in data_list
])
print(
    "%d out of %d records are corrected by transponders." % (
        transponder_correction_filter.sum(),
        len(transponder_correction_filter)
    )
)

In [ ]:
TRANS_MAP_DIR_SUFFIX = "agv_ns_ros/data/AGV_Map/"
TRANS_MAP_DIR = ""  # leave this empty to enable auto search

CWD = os.getcwd()

if not TRANS_MAP_DIR:
    current_dir = os.path.dirname(CWD)
    while (parent_dir := os.path.dirname(current_dir)) != current_dir:
        current_dir = parent_dir
        map_dir_path = os.path.join(current_dir, TRANS_MAP_DIR_SUFFIX)
        if os.path.exists(map_dir_path):
            TRANS_MAP_DIR = map_dir_path
            break
    else:
        raise FileNotFoundError("Failed to find the transponder map file!")

trans_map_candidates = [
    file_name
    for file_name in os.listdir(TRANS_MAP_DIR)
    if file_name.startswith("TransMap") and file_name.endswith(".csv")
]
if len(trans_map_candidates) < 1:
    raise FileNotFoundError("Transponder map file not found!")
else:
    selected_trans_map_file_name = sorted(trans_map_candidates)[-1]

TRANS_MAP_PATH = os.path.join(TRANS_MAP_DIR, selected_trans_map_file_name)
print(f'Selected "{os.path.relpath(TRANS_MAP_PATH, CWD)}".')
df_trans_map = pd.read_csv(
    TRANS_MAP_PATH,
    names=["TransID", "AbsX", "AbsY", "LaneNB1", "LaneNB2"],
    index_col=0,
)
df_trans_map["x"] = df_trans_map["AbsX"] / 1000
df_trans_map["y"] = df_trans_map["AbsY"] / 1000


def plot_transponders(ax):
    handle = None
    for id in df_trans_map.index:
        handle = ax.plot(
            df_trans_map.loc[id, "x"],
            df_trans_map.loc[id, "y"],
            "+",
            ms=5,
            color="purple",
            alpha=0.5,
            label=("Transponders" if handle is None else None),
        )[0]
    return handle

## ENU

In [ ]:
x_enu = np.array(data_dict["enu_x"])
y_enu = np.array(data_dict["enu_y"])
z_enu = np.array(data_dict["enu_z"])

fig = plt.figure(figsize=(6, 6), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.plot(x_enu, y_enu, "b.-", ms=2, alpha=0.5, label="ENU")
ax.plot(x_enu[0], y_enu[0], "ro", ms=8, alpha=0.5, label="ENU Start")

ax.set(
    xlabel="x",
    xlim=(x_enu.min() - 5, x_enu.max() + 5),
    ylabel="y",
    ylim=(y_enu.min() - 5, y_enu.max() + 5),
    aspect="equal",
)
ax.legend()
ax.grid(alpha=0.3)

## Center Position By GNSS

In [ ]:
x_gnss = np.array(data_dict["gnss_X"])
y_gnss = np.array(data_dict["gnss_Y"])

fig = plt.figure(figsize=(6, 6), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.plot(x_gnss, y_gnss, "b.-", ms=2, alpha=0.5, label="GNSS")
ax.plot(x_gnss[0], y_gnss[0], "ro", ms=8, alpha=0.5, label="GNSS Start")

plot_transponders(ax)

ax.set(
    xlabel="x",
    xlim=(x_gnss.min() - 5, x_gnss.max() + 5),
    ylabel="y",
    ylim=(y_gnss.min() - 5, y_gnss.max() + 5),
    aspect="equal",
)
ax.legend()
ax.grid(alpha=0.3)

## Recomputed Center Position By GNSS

In [ ]:
INS_OFFSET_X = 0.313919334029
INS_OFFSET_Y = 2.814034003987

cos_heading = np.cos(heading)
sin_heading = np.sin(heading)
x_recomputed = 687.536114629993 \
    + INS_OFFSET_X * -cos_heading \
    + INS_OFFSET_Y * sin_heading \
    + 0.789918276523 * x_enu \
    + 0.609474825749 * y_enu \
    + -0.236295772317 * z_enu \
    + -0.000020183664 * (x_enu * y_enu) \
    + -0.005486946410 * (y_enu * z_enu) \
    + 0.003718934791 * (z_enu * x_enu)
y_recomputed = 108.596183544605 \
    + INS_OFFSET_X * -sin_heading \
    + INS_OFFSET_Y * -cos_heading \
    + -0.618264990381 * x_enu \
    + 0.788027481693 * y_enu \
    + 0.234559052464 * z_enu \
    + 0.000109658679 * (x_enu * y_enu) \
    + 0.003845675972 * (y_enu * z_enu) \
    + -0.000196675328 * (z_enu * x_enu)

fig = plt.figure(figsize=(6, 6), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.plot(x_recomputed, y_recomputed, "b.-",
        ms=2, alpha=0.5, label="Recomputed")
ax.plot(x_recomputed[0], y_recomputed[0], "ro",
        ms=8, alpha=0.5, label="Recomputed Start")

plot_transponders(ax)

ax.set(
    xlabel="x",
    xlim=(x_recomputed.min() - 5, x_recomputed.max() + 5),
    ylabel="y",
    ylim=(y_recomputed.min() - 5, y_recomputed.max() + 5),
    aspect="equal",
)
ax.legend()
ax.grid(alpha=0.3)